In [1]:
# LLM Call architecture

# 1. Creating a StateGraph
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    # Messages have the type "list". The `add_messages` 
    # function in the annotation defines how this state should 
    # be updated (in this case, it appends new messages to the 
    # list, rather than replacing the previous messages)
    messages: Annotated[list, add_messages]
    
builder = StateGraph(State)

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
def chatbot(state: State):
    answer = model.invoke(state["messages"])
    return {"messages": [answer]}

# The first argument is the unique node name
# The second argument is the function or Runnable to run
builder.add_node("chatbot", chatbot)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [3]:
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)
graph = builder.compile()

graph.get_graph().draw_mermaid_png()

ValueError: Found edge starting at unknown node 'chatbot'

In [ ]:
from langchain_core.messages import HumanMessage

input = {"messages": [HumanMessage(content="hi!")]}
                                   
for chunk in graph.stream(input):
    print(chunk)

# { "chatbot": { "messages": [AIMessage("How can I help you?")] } }

In [ ]:
# langGraph with built-in persistence

from langgraph.checkpoint.memory import MemorySaver
graph = builder.compile(checkpointer=MemorySaver())

In [ ]:
result_1 = graph.invoke({ "messages": [HumanMessage(content="hi, my name is Jack!")] })
# { "chatbot": { "messages": [AIMessage("How can I help you Jack?")] } }
result_2 = graph.invoke({ "messages": [HumanMessage(content="what is my name?")] })
# { "chatbot": { "messages": [AIMessage("Your name is Jack")] } }